# Batch normalization , Layer Normalization,  Instance Normalization, and Group Normalization


![image](https://github.com/OC-JSPark/oc-jspark.github.io/assets/46878973/82439224-d699-46a3-ba82-d439bd5dd0f6)

https://wandb.ai/wandb_fc/GroupNorm/reports/Group-Normalization-in-Pytorch-With-Examples---VmlldzoxMzU0MzMy



https://pytorch.org/docs/stable/generated/torch.nn.GroupNorm.html

resnet에서 성능향상에 중요한 영향을 준 batch normalization 과 variation에 대해서 확인해보기

batch normalization : batch 기반의 normalize기법으로 internal covariance shift덕분에 잘된다. 그러나 추후에는 optimize randscape를 smoothing시켜줘서 안정적인 학습을 가능하게 해줌.
단점 : batch size에 민감함. batch size가 32,256,1024를 쓸때는 잘됨. 그러나 그 이하일떄는 size가 줄어들수록 성능이 매우 크게 감소함. 
그래서 대안으로 나온게 layer norm, instance norm, group norm이 있다.
N : batch
C : channel
W : image width size

batch norm : 한채널에 대해서 normalize가 각각되는것.
layer norm : 모든 channel을 한번에 normalize한다.하나의 instance에 대해서 normalize한다, input size가 제한이 되어있을떄(RNN같은구조에서는 sequence가 계속 늘어날수있음.) 성능이 좋을수있다. RNN에서 layer norm을 그래서 주로 쓴다. 해당 논문보면 batch norm은 RNN에서 잘안된다.
instance norm : 채널 하나에 대해서 normalize한다. 즉, batch norm과 똑같지만, instance 단위로 되는게 다른점, batch가 중요하지 않은경우 매우 효과적임.image match같은경우, 혹은 batch 간에 corelation 이 없는경우 매우 효과적임.예를들면 optical flow같은 문제가 있는경우 이미지 2장사이에서 차이를 구해야하는데 차이를 구할때 pair1과 pair2가 관계가 없을수도 있다. 이럴때 instance norm을 쓰는게 좋다. 
* image classification은 batch사이에서 corelation이 잇다.  두 장 사이에 instance를 고려해줘야 하기때문에 batch norm을 쓰는게 효과적이지만 그렇지 않은경우에 batch norm을 쓰면 코릴레이션도 없는데 같이 normalize해주면 안좋은결과를 줄수있다.

group norm : 그룹을 나눠서 그룹별로 normalization한다, batch size가 작아질때 group norm 보다 batch norm error가 엄청커진다. group norm은 안정적이다.

https://pytorch.org/docs/stable/nn.html#normalization-layers
여기서 normalization layers로 들어가서 각 해당 normalize에 대한 논문을볼수있다.

In [1]:
import torch, torchvision
import torch.nn as nn
import torchvision.models as models
import torchvision.datasets as datasets

import matplotlib.pyplot as plt
from PIL import Image


Batch norm, layer norm, instance norm, group norm

https://pytorch.org/docs/stable/nn.html





https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html#torch.nn.LayerNorm

https://pytorch.org/docs/stable/generated/torch.nn.GroupNorm.html
    

In [3]:
## batchnorm에서는 2차원 tensor에 대해서 n,c,hw에 대해서 채널만 주면 normalize해주는것.
## 이걸해주면 instance를 만들어진다. 이게 핵심임. 
## layernorm도 input shape이 먼지 넣어주면 끝.
## input channel과 output channel이 어떻게 나올지 network를 추가로 만들어준것임
## group norm은 group 기반으로 한다. channel 1024가 있으면 group 2개라고 한다면
## 512개씩 나눠서 가질테니깐 그룹사이즈를 미리 정해줘야 한다.
## 그래서 group size까지 parameter로 받아주는게 해당 클래스의 특징이다.

class BatchNorm(nn.Module):
    def __init__(self, in_channel, out_channels):
        super(BatchNorm, self).__init__()
        self.bn = nn.BatchNorm2d(in_channel)
        
    def forward(self,x):
        out = self.bn(x)  #[N, C, HW] -> [N, C, HW]
        
        
        return out

    
## For different sequences, e.g., RNN.
class LayerNorm(nn.Module):
    def __init__(self, in_shape, out_channels):
        super(LayerNorm, self).__init__()
        ## 아래처럼 추가로 만들어줘도 된다.
        ## self.conv = nn.Conv2D(in_shape, out_shape, kernel, stride,dlate)
        self.ln = nn.LayerNorm(in_shape, eps=1e-08)

    def forward(self,x):
        out = self.ln(x)  #[N, C, HW] -> [N, C, HW]

        
        return out

    
## For style transfer, domain adaptation.
class InstanceNorm(nn.Module):
    def __init__(self, in_channel, out_channels):
        super(InstanceNorm, self).__init__()
        self.In = nn.InstanceNorm2d(in_channel, eps=1e-08) 

    def forward(self,x):
        out = self.In(x)  #[N, C, HW] -> [N, C, HW]
        return out

    
## stable in small batch size.
class GroupNorm(nn.Module):
    def __init__(self, group_size, in_channel, out_channels):
        super(GroupNorm, self).__init__()
        self.gn = nn.GroupNorm(group_size, in_channel, eps=1e-08)  ## num_group and in_channel

    def forward(self,x):
        out = self.gn(x) #[N, C, HW] -> [N, C, HW]
        
        return out


In [6]:
in_channel = 64
feature = torch.randn(8, in_channel, 120, 120)  ## temp tensor [B, C, H, W]

## batchnorm layer만들어주고 batchnorm 통과시켜주면
BN = BatchNorm(in_channel, out_channels=64)
## output feature가 나오게 된다.
out_feat = BN(feature)

print(out_feat.shape)       ## normalize된 output이며, shape은 똑같고, batch 단위로 전체가 normalize된것
                            ## 8개 한번에 normalize된것.

torch.Size([8, 64, 120, 120])


In [8]:
## input shape이 batch단위로 되는게 아니다.
## 이전에는 input channel인데 layernorm은 input shape을 넣어줘야한다.
LN = LayerNorm(in_shape=list(feature.shape[1:]), out_channels=64)

out_feat = LN(feature)

print(out_feat.shape)   ## 8개. 즉 각각이 따로 normalize되는것.

torch.Size([8, 64, 120, 120])


In [9]:
IN=InstanceNorm(in_channel, out_channels=64)

out_feat = IN(feature)

print(out_feat.shape)   ## 8,64를 제외하고 120이 한번에 normalize되는것.

torch.Size([8, 64, 120, 120])


In [10]:
GN=GroupNorm(group_size=2, in_channel=in_channel, out_channels=64)

out_feat = GN(feature)

print(out_feat.shape)  ## 32 / 32 : 32개씩 나눠진 채널들이 그룹normalize된다.

GN=GroupNorm(group_size=4, in_channel=in_channel, out_channels=64)

out_feat = GN(feature)

print(out_feat.shape)  ## 16 / 16 / 16 / 16 : 16개씩 나눠진것.

torch.Size([8, 64, 120, 120])
torch.Size([8, 64, 120, 120])
